<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer による自然言語処理

### 準備

関連ライブラリのインストール

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q datasets

関連ライブラリのインポート

In [ ]:
import torch
import textwrap
from transformers import T5Tokenizer, BertTokenizer
from transformers import AutoModelForCausalLM, RobertaForMaskedLM
from transformers import BertForPreTraining

## 1. 日本語モデルによる文章中の空欄埋め





### モデルのダウンロード

In [ ]:
# トーカナイザの設定
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading
# モデルの設定
model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")

### 問題文の作成

In [ ]:
# 原文
text = "4年に1度オリンピックは開かれる。"

# 文頭に [CLS] を付加
text = "[CLS]" + text + "[SEP]"

# トークン化
tokens = tokenizer.tokenize(text)
print(tokens)

# トークンにマスクをかける
masked_idx = 5
tokens[masked_idx] = tokenizer.mask_token
print(tokens) 

### 穴埋め問題を解く

補充すべき単語の推定 (id)

In [ ]:
# トークンから単語 ID に変換
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

# テンソルに変換
token_tensor = torch.LongTensor([token_ids])

結果の表示

空欄に当てはまると思われる単語を確信の高い順に複数出力します。

In [ ]:
# 場所の ID を与える
position_ids = list(range(0, token_tensor.size(1)))
position_id_tensor = torch.LongTensor([position_ids])

# マスクされたトークンの予測値（トップ 10）
with torch.no_grad():
    outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

コンピュータが出してきた結果が妥当であるか、確認してください。

## 2. 英語モデルによる TOEIC Part 5 の解答

上記の穴埋めを英語の文章でやってみます。

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

### 問題文の作成

問題文：text、選択肢：candidate

In [ ]:
text = "Customer reviews indicate that many modern mobile devices are often unnecessarily [MASK] ."
candidate = ["complication", "complicates", "complicate", "complicated"]

BERTに分かるように変換 (text --> tokens)

In [ ]:
tokens = tokenizer.tokenize(text)
print(tokens)
masked_index = tokens.index("[MASK]")
tokens = ["[CLS]"] + tokens + ["[SEP]"]

print(tokens)
print(masked_index)

### 解答の作成

日本語のように穴埋めでやったように候補をたくさん出力し、選択肢と一致する最初の単語を問題の答とします。

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids = torch.tensor(ids).reshape(1,-1)  # バッチサイズ1の形に整形
predictions = model(ids)[0][0]
print(predictions)

In [ ]:
_, predicted_indexes = torch.topk(predictions[masked_index+1], k=1000)
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
# -> ['expensive', 'small', 'priced', 'used', ...
print(predicted_tokens)

In [ ]:
for i, v in enumerate(predicted_tokens):
    if v in candidate:
        print(i, v)
        break

得られた答えが自分で考えた答えと一致したか確認してください。

### 解答の作成を関数にする

ここまで一連の流れを関数にして、簡単に実行できるようにします。

In [ ]:
def part5_slover(text, candidate):
    tokens = tokenizer.tokenize(text)
    masked_index = tokens.index("[MASK]")
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

    ids = tokenizer.convert_tokens_to_ids(tokens)
    ids = torch.tensor(ids).reshape(1,-1)
    predictions = model(ids)[0][0]

    _, predicted_indexes = torch.topk(predictions[masked_index+1], k=10000)
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())

    for i, v in enumerate(predicted_tokens):
        if v in candidate:
            return "answer: " + v
    return "don't know"

### 練習問題で試してみる
上記は、公式サンプル問題 --> https://www.iibc-global.org/toeic/test/lr/about/format/sample05.html　からの出題でした。残りが４問あるので、まずは自分で答えてみてください。

以下では、残りの問題をコンピュータに答えさせます。

第２問〜第５問（問題文：text2〜5、選択肢：candidate2〜5）の入力

In [ ]:
text2 = "Jamal Nawzad has received top performance reviews [MASK] he joined the sales department two years ago ."
candidate2 = ["despite", "except", "since", "during"]
text3 = "Gyeon Corporation’s continuing education policy states that [MASK] learning new skills enhances creativity and focus ."
candidate3 = ["regular", "regularity", "regulate", "regularly"]
text4 = "Among [MASK] recognized at the company awards ceremony were senior business analyst Natalie Obi and sales associate Peter Comeau. ."
candidate4 = ["who", "whose", "they", "those"]
text5 = "All clothing sold in Develyn’s Boutique is made from natural materials and contains no [MASK] dyes ."
candidate5 = ["immediate", "synthetic", "reasonable", "assumed"]

解答の出力

In [ ]:
print(part5_slover(text2, candidate2))
print(part5_slover(text3, candidate3))
print(part5_slover(text4, candidate4))
print(part5_slover(text5, candidate5))

## 3. GPT-2 による文書生成
GTP-2 の日本語モデルは複数あります。本実験では rinna のデータを用いたものを使います。

### (1) rinna/japanese-gpt2 の利用

### モデルのダウンロード

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

### 文書生成の例

防大の志望動機を書かせてみます。

In [ ]:
input = tokenizer.encode("私が防衛大学校を志望したのは、", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=200, num_return_sequences=8)
sentences = tokenizer.batch_decode(output)
print("=========================================================")
for i in sentences:
    print(textwrap.fill(i.replace('</s> ', ''), 50))
    print("=========================================================")

### 文体の変更（モデルのチューニング）

人工知能が出力する文章は、学習に使ったデータ次第で変わります。

ここで、学内メールに添付されているされている train.txt と run_clm.py をアップロードしてください（画面左のファイルアイコンを押すとファイル画面がでますので、そこにドラッグ＆ドロップでアップロードできます）。

アップロードができたら下の命令を実行してください。

In [ ]:
%%time
!rm -r output

# ファインチューニングの実行
!python ./run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-small \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=30 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --output_dir=output/ \
    --use_fast_tokenizer=False

同じ書き出しで文章を生成します。

In [ ]:
# モデルの準備
model2 = AutoModelForCausalLM.from_pretrained("output/")

# 推論
input = tokenizer.encode("私が防衛大学校を志望したのは、", return_tensors="pt")
output = model2.generate(input, do_sample=True, max_length=200, num_return_sequences=8)
sentences = tokenizer.batch_decode(output)
print("=========================================================")
for i in sentences:
    print(textwrap.fill(i.replace('</s>', ''), 50))
    print("=========================================================")

文体が変わったかどうかを確認してください。

### 課題：自分のデータでモデルのチューニング

train.txt に自分で用意した文章データを保存し、再度アップロードしてください。

新データで学習（チューニング）をします。

In [ ]:
%%time
!rm -r output

# ファインチューニングの実行
!python ./run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-small \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=30 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --output_dir=output/ \
    --use_fast_tokenizer=False

文章の生成：書き出しが「XXXX」となっているので自分のデータに合わせた書き出しにして実行してください。

In [ ]:
# モデルの準備
model2 = AutoModelForCausalLM.from_pretrained("output/")

# 推論
input = tokenizer.encode("XXXX、", return_tensors="pt")
output = model2.generate(input, do_sample=True, max_length=200, num_return_sequences=8)
sentences = tokenizer.batch_decode(output)
print("=========================================================")
for i in sentences:
    print(textwrap.fill(i.replace('</s>', ''), 50))
    print("=========================================================")

どのような文章が生成されたか、またチューニングがうまく行ったか、行かなかった場合にはどのような原因が考えられるか、などについて考察してください。